# NLP: scikit-learn basics

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/phonchi/ModularPython/blob/master/NLP-scikit_learn_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/phonchi/ModularPython/blob/master/NLP-scikit_learn_basics.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

![Creative Commons License](https://i.creativecommons.org/l/by/4.0/88x31.png)  
This work by Jephian Lin is licensed under a [Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0/).

In [ ]:
import numpy as np
import pandas as pd

## Scikit-learn API

### Utilizing Scikit-learn for Machine Learning Models

The `scikit-learn` (or `sklearn`) library is a powerful tool for machine learning, offering a wide range of ready-to-use models and datasets. The algorithms are implemented in a straightforward manner with minimal heuristic modifications, making it ideal for educational purposes. One of the key strengths of `scikit-learn` is its consistent API, which simplifies the process of using different models. The typical workflow for utilizing a model in `scikit-learn` can be summarized as follows:

```python
from sklearn.decomposition import PCA  # Import the model
model = PCA(2)              # Initialize the model with hyperparameters
X_new = model.fit_transform(X)      # Fit the model to the data and apply transformation
```

**Terminologies**:
- **Data:** A collection of information that may be numerical, textual, visual, or auditory.
- **Label:** The target output for each data sample, such as a product review score.
- **Model:** A mathematical function used for making predictions or transformations, derived from applying an algorithm to data.
- **Fit or Train:** The process of adjusting the model's parameters based on the data.
- **Transformation:** The conversion of data into a new format or structure using the model.
- **Prediction:** Estimating outputs based on input data using the model.

We will use the following data to demonstrate two algorithms, PCA and $k$-means.

In [ ]:
import nltk
nltk.download('inaugural')

from nltk.corpus import inaugural

files = inaugural.fileids()
texts = [inaugural.raw(file) for file in files]
years = [file[:-4].split("-")[0] for file in files]
presidents = [file[:-4].split("-")[1] for file in files]
df = pd.DataFrame({
    "year": years,
    "president": presidents,
    "file": files,
    "text": texts
})
df.set_index("year", inplace=True)
df.tail() # print last few files

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
X = tfidf.fit_transform(texts).toarray()
print(X.shape)

### Principal component analysis (PCA)

In our data, each sample is a vector of 8,984 entries.  We usually consider it as a point in an 8,984-dimensional space.  However, it is almost impossible for us to see such a high-dimensional space.  We will transform the data into lower dimension, with the minimum loss of information.  

PCA is a dimensionality reduction algorithm.

In [ ]:
from sklearn.decomposition import PCA

model = PCA(n_components=10)
X_new = model.fit_transform(X)
X_new.shape

In [ ]:
import matplotlib.pyplot as plt
exp_var_pca = model.explained_variance_ratio_
#
# Cumulative sum of eigenvalues; This will be used to create step plot
# for visualizing the variance explained by each principal component.
#
cum_sum_eigenvalues = np.cumsum(exp_var_pca)
#
# Create the visualization plot
#
plt.bar(range(0,len(exp_var_pca)), exp_var_pca, alpha=0.5, align='center', label='Individual explained variance')
plt.step(range(0,len(cum_sum_eigenvalues)), cum_sum_eigenvalues, where='mid',label='Cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal component index')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

In [ ]:
df['x0'] = X_new[:,0]
df['x1'] = X_new[:,1]
df.plot(kind='scatter', x='x0', y='x1',
    color=df.index.astype(int), hover_data=['president'],
    backend='plotly')

### $k$-means clustering

When a dataset is given, one may try to partition the data points into several clusters and predict their labels.

The $k$-means clustering algorithm will give a "reasonable" clustering label to each point.

_The code below use the same data as above and use the PCA-transformed feature to plot the points._

In [ ]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters=3, n_init='auto')
y = model.fit_predict(X)

In [ ]:
df['y'] = y.astype('object')
df.plot(kind='scatter', x='x0', y='x1',
        color='y', hover_data=[df.index, 'president'],
        backend='plotly')

## 🔍 Supplementary: `scikit-ollama`

It allows as to seamlessly integrate powerful language models like ChatGPT into scikit-learn for enhanced text analysis tasks.

In [ ]:
!pip install scikit-ollama -qq
!pip install colab-xterm -qq
!apt install lshw
!curl -fsSL https://ollama.com/install.sh | sh

Paste the following commands to the terminal:

```
export OLLAMA_MAX_LOADED_MODELS=2 # sets the max number of loaded models
export OLLAMA_NUM_PARALLEL=2 # sets the max number of parallel tasks
ollama serve

```

In [ ]:
%load_ext colabxterm

In [ ]:
%xterm height=500

In [ ]:
%%capture

!ollama pull llama3
!ollama pull nomic-embed-text

In [ ]:
!ollama pull gte-large-en-v1.5

In [ ]:
from skllm.config import SKLLMConfig

SKLLMConfig.set_gpt_url("http://127.0.0.1:11434/")

In [ ]:
import string
# Convert each text to lowercase and remove specific characters
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove new line and tab characters
    text = text.replace('\n', '').replace('\t', '')
    # Remove punctuation
    for p in string.punctuation:
        text = text.replace(p, '')
    return text

# Apply preprocessing to each text
texts = [preprocess_text(text) for text in texts]

# Display the processed texts
print(texts)

In [ ]:
from skollama.models.ollama.vectorization import OllamaVectorizer

vectorizer = OllamaVectorizer(batch_size=2) # batch_size is number of parallel tasks
X_vec = vectorizer.fit_transform(texts)
print(X_vec.shape)

In [ ]:
from sklearn.decomposition import PCA
#from sklearn.preprocessing import StandardScaler

#sc = StandardScaler()
#X_vec = sc.fit_transform(X_vec)
model = PCA(10)
X_new = model.fit_transform(X_vec)
X_new.shape

In [ ]:
import matplotlib.pyplot as plt
exp_var_pca = model.explained_variance_ratio_
#
# Cumulative sum of eigenvalues; This will be used to create step plot
# for visualizing the variance explained by each principal component.
#
cum_sum_eigenvalues = np.cumsum(exp_var_pca)
#
# Create the visualization plot
#
plt.bar(range(0,len(exp_var_pca)), exp_var_pca, alpha=0.5, align='center', label='Individual explained variance')
plt.step(range(0,len(cum_sum_eigenvalues)), cum_sum_eigenvalues, where='mid',label='Cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal component index')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

In [ ]:
df['x0'] = X_new[:,0]
df['x1'] = X_new[:,1]
df.plot(kind='scatter', x='x0', y='x1',
    color=df.index.astype(int), hover_data=['president'],
    backend='plotly')

In [ ]:
from skllm.datasets import get_classification_dataset
from skollama.models.ollama.classification.zero_shot import ZeroShotOllamaClassifier

# Load a demo dataset
X, y = get_classification_dataset()

# Initialize the model and make the predictions
clf = ZeroShotOllamaClassifier(model="llama3")
clf.fit(None, ["positive", "negative", "neutral"])
#clf.fit(X, y)
labels = clf.predict(X)

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "Text": X,
    "Predicted Label": labels,
    "True Label": y
})
df

> Checkout [here](https://ollama.com/library) for more models. Also [here](https://andreaskarasenko.github.io/skollama-docs/) for more information about `scikit-ollama`.

### 📚 Further reading

- [_Python Data Science Handbook_](https://jakevdp.github.io/PythonDataScienceHandbook/) by Jake VanderPlas